In [22]:
import os
from dotenv import load_dotenv, find_dotenv

In [23]:
load_dotenv(find_dotenv())
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [24]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

## Comma Separated List (List parser)

In [25]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [26]:
output_parser = CommaSeparatedListOutputParser()

In [27]:
format_instructions = output_parser.get_format_instructions()

In [28]:
prompt = PromptTemplate(
    template="provide 5 examples of {query}. \n{format_instructions}",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions}
)

In [31]:
llm = OpenAI(temperature=.9, model="text-davinci-003")

In [32]:
prompt = prompt.format(query="Currencies")
output = llm(prompt)

In [33]:
print(output)



US Dollar, Euro, British Pound, Japanese Yen, Indian Rupee


## JSON Format

In [38]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [39]:
response_schemas = [
    ResponseSchema(name="currency", description="answer to the user's question"),
    ResponseSchema(name="abbrevation", description="What's the abbrebation of that currency")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [40]:
print(output_parser)

response_schemas=[ResponseSchema(name='currency', description="answer to the user's question", type='string'), ResponseSchema(name='abbrevation', description="What's the abbrebation of that currency", type='string')]


In [41]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"currency": string  // answer to the user's question
	"abbrevation": string  // What's the abbrebation of that currency
}
```


In [57]:
prompt = PromptTemplate(
    template="Answer the users question as best as possible. \n{format_instructions}\n{query}",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions}
)
print(prompt)

input_variables=['query'] output_parser=None partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"currency": string  // answer to the user\'s question\n\t"abbrevation": string  // What\'s the abbrebation of that currency\n}\n```'} template='Answer the users question as best as possible. \n{format_instructions}\n{query}' template_format='f-string' validate_template=True


In [58]:
prompt = prompt.format(query="What's the currency of Japan?")

In [59]:
output = llm(prompt)
print(output)



```json
{
	"currency": "Japanese Yen",
	"abbreviation": "JPY"
}
```


'\n\n```json\n{\n\t"currency": "Japanese Yen",\n\t"abbreviation": "JPY"\n}\n```'

## Pydantic (JSON) parser

In [60]:
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [61]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [65]:
llm = OpenAI(model_name="text-davinci-003", temperature=0)

In [67]:
# Here's another example, but with a compound typed field.
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")


actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query=actor_query)

output = llm(_input.to_string())

parser.parse(output)

Actor(name='John Doe', film_names=['The Great Escape', 'The Godfather', 'The Shawshank Redemption', 'The Dark Knight'])

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "description": "name of an actor", "type": "string"}, "film_names": {"title": "Film Names", "description": "list of names of films they starred in", "type": "array", "items": {"type": "string"}}}, "required": ["name", "film_names"]}\n```'